In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Load raw dataset paths
DATA_DIR = Path("../../dataset/raw")

In [2]:
gold_df = pd.read_csv(DATA_DIR / "gold_historical_data.csv")
usd_df = pd.read_csv(DATA_DIR / "dx_y_historical_data.csv")
cpi_df = pd.read_csv(DATA_DIR / "CPIAUCSL.csv")
yield_df = pd.read_csv(DATA_DIR / "DGS10.csv")
guardian_df = pd.read_csv(DATA_DIR / "news_sentiment.csv")

# Preview the datasets
gold_df.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Dec 30, 2024","2,620.70","2,626.90","2,597.00","2,606.10","2,606.10",794
1,"Dec 27, 2024","2,617.70","2,617.70","2,616.40","2,617.20","2,617.20",642
2,"Dec 26, 2024","2,628.50","2,638.80","2,627.90","2,638.80","2,638.80",84
3,"Dec 24, 2024","2,613.00","2,620.00","2,609.50","2,620.00","2,620.00",35
4,"Dec 23, 2024","2,620.00","2,627.70","2,611.10","2,612.30","2,612.30",451


In [3]:
gold_df["Date"] = pd.to_datetime(gold_df["Date"])
usd_df["Date"] = pd.to_datetime(usd_df["Date"])
cpi_df["DATE"] = pd.to_datetime(cpi_df["observation_date"])
yield_df["DATE"] = pd.to_datetime(yield_df["observation_date"])
guardian_df["Date"] = pd.to_datetime(guardian_df["date"]).dt.normalize()

In [4]:
# --- Clean gold_df ---
gold_df.columns = gold_df.columns.str.replace("*", "", regex=False).str.replace("**", "", regex=False).str.strip()
gold_df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
for col in ["Open", "High", "Low", "Close", "Adj_Close"]:
    gold_df[col] = pd.to_numeric(gold_df[col].astype(str).str.replace(",", ""), errors="coerce")

# --- Clean usd_df ---
usd_df.columns = usd_df.columns.str.replace("*", "", regex=False).str.replace("**", "", regex=False).str.strip()
usd_df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
for col in ["Open", "High", "Low", "Close", "Adj_Close"]:
    usd_df[col] = pd.to_numeric(usd_df[col].astype(str).str.replace(",", ""), errors="coerce")



# --- Clean cpi_df ---
cpi_df.rename(columns={"observation_date": "Date", "CPIAUCSL": "CPI"}, inplace=True)
cpi_df["CPI"] = pd.to_numeric(cpi_df["CPI"], errors="coerce")

# --- Clean yield_df ---
yield_df.rename(columns={"observation_date": "Date", "DGS10": "Yield_10Y"}, inplace=True)
yield_df["Yield_10Y"] = pd.to_numeric(yield_df["Yield_10Y"], errors="coerce")


In [5]:
# --- Ensure all Date columns are datetime64[ns] and normalized ---
gold_df["Date"] = pd.to_datetime(gold_df["Date"]).dt.normalize()
usd_df["Date"] = pd.to_datetime(usd_df["Date"]).dt.normalize()
cpi_df["Date"] = pd.to_datetime(cpi_df["Date"]).dt.normalize()
yield_df["Date"] = pd.to_datetime(yield_df["Date"]).dt.normalize()
guardian_df["Date"] = pd.to_datetime(guardian_df["Date"]).dt.normalize()  # sentiment-based one

In [6]:
# Start with gold prices as the base
merged_df = gold_df.copy()

# Merge in USD Index
merged_df = pd.merge(merged_df, usd_df[["Date", "Adj_Close"]].rename(columns={"Adj_Close": "USD_Index"}), on="Date", how="left")

# Merge in CPI
merged_df = pd.merge(merged_df, cpi_df, on="Date", how="left")

# Merge in Treasury Yield
merged_df = pd.merge(merged_df, yield_df, on="Date", how="left")


In [7]:
# Sort by date just to be safe
merged_df.sort_values("Date", inplace=True)

# ✅ Forward fill monthly CPI and Yield values (non-deprecated)
merged_df[["CPI", "Yield_10Y"]] = merged_df[["CPI", "Yield_10Y"]].ffill()

# (Optional) Backfill beginning if needed
merged_df[["CPI", "Yield_10Y"]] = merged_df[["CPI", "Yield_10Y"]].bfill()

In [8]:
from pathlib import Path
from datetime import datetime

# Create output folder if it doesn't exist
output_dir = Path("../../dataset/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Generate filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = output_dir / f"merged_gold_macro_{timestamp}.csv"

# Save the DataFrame
merged_df.to_csv(output_path, index=False)
print(f"✅ Merged dataset saved to: {output_path}")

✅ Merged dataset saved to: ../../dataset/processed/merged_gold_macro_20250331_162426.csv


In [9]:
# --- Clean and prepare pre-aggregated Guardian sentiment data ---
guardian_df["Date"] = pd.to_datetime(guardian_df["Date"]).dt.normalize()

# --- Merge into merged_df ---
merged_df = pd.merge(merged_df, guardian_df, on="Date", how="left")

# Fill missing sentiment values
merged_df["headline_count"] = merged_df["headline_count"].fillna(0).astype(int)
merged_df["avg_sentiment"] = merged_df["avg_sentiment"].fillna(0)
merged_df["std_sentiment"] = merged_df["std_sentiment"].fillna(0)

In [13]:
# --- Drop unnecessary date-related columns ---
merged_df.drop(columns=["DATE_x", "DATE_y", "date"], inplace=True, errors="ignore")

In [14]:
from pathlib import Path
from datetime import datetime

# Create output folder if it doesn't exist
output_dir = Path("../../dataset/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Generate filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = output_dir / f"merged_gold_macro_news_{timestamp}.csv"

# Save the DataFrame
merged_df.to_csv(output_path, index=False)
print(f"✅ Merged dataset with Guardian news saved to: {output_path}")

✅ Merged dataset with Guardian news saved to: ../../dataset/processed/merged_gold_macro_news_20250331_163023.csv


In [15]:
# Optional: Save also as the "final" version for modeling/EDA
final_output_path = output_dir / "merged_gold_macro_final.csv"
merged_df.to_csv(final_output_path, index=False)
print(f"🟢 Final version saved to: {final_output_path}")

🟢 Final version saved to: ../../dataset/processed/merged_gold_macro_final.csv
